# Module 3: Hybrid RAG

We already went through the vanilla RAG in the Module 1. Here we will introduce a new type of RAG, which is Hybrid RAG. In some senarios, we want a higher recall rate, but the embedding vector search cannot offer a good performance.

In this case, we will usually use multi-channel retrieval to improve the performance. And the another channel aside from the embedding vector search is the BM25.

This module uses Amazon Bedrock Embedding model and TiDB Serverless Vector Search.

> **Note:**
>
> - We already set the `SERVERLESS_CLUSTER_HOST`, `SERVERLESS_CLUSTER_PORT`, `SERVERLESS_CLUSTER_USERNAME`, `SERVERLESS_CLUSTER_PASSWORD`, and `SERVERLESS_CLUSTER_DATABASE_NAME` in the environment parameters.
> - We also granted the permission of using Amazon Bedrock for this lab. If you want to use this code snippet out of TiDB Labs platform, please set them beforehand.

## Install Dependencies

In [ ]:
%pip install -q \
    pytidb==0.0.10.dev1 \
    boto3==1.38.23 \
    litellm \
    pandas

## Initial the Clients of Database

In [ ]:
import os

from litellm import completion
from typing import Optional, Any
from pytidb import TiDBClient
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction

db = TiDBClient.connect(
    host=os.getenv("SERVERLESS_CLUSTER_HOST"),
    port=int(os.getenv("SERVERLESS_CLUSTER_PORT")),
    username=os.getenv("SERVERLESS_CLUSTER_USERNAME"),
    password=os.getenv("SERVERLESS_CLUSTER_PASSWORD"),
    database=os.getenv("SERVERLESS_CLUSTER_DATABASE_NAME"),
    enable_ssl=True,
)

embedding_model = "bedrock/amazon.titan-embed-text-v2:0"

text_embedding_function = EmbeddingFunction(
    embedding_model,
    timeout=60
)

## Prepare the Context

Just like the Module 1, we created the table `documents`, but this time, we added a new index by using this line:

```python
table.create_fts_index("text")
```

It creates a new Full-text Search Index based on the `text` column.

In [ ]:
table_name = "documents"
class Document(TableModel, table=True):
    __tablename__ = table_name
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    text: str = Field(max_length=1024)
    embedding: Optional[Any] = text_embedding_function.VectorField(
        source_field="text",
    )

documents = [
    Document(id=1, text="TiDB is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads."),
    Document(id=2, text="TiFlash is the key component that makes TiDB essentially an Hybrid Transactional/Analytical Processing (HTAP) database. As a columnar storage extension of TiKV, TiFlash provides both good isolation level and strong consistency guarantee."),
    Document(id=3, text="TiKV is a distributed and transactional key-value database, which provides transactional APIs with ACID compliance. With the implementation of the Raft consensus algorithm and consensus state stored in RocksDB, TiKV guarantees data consistency between multiple replicas and high availability. "),
]

table = db.create_table(schema=Document, if_exists="overwrite")

table.create_fts_index("text")

table.bulk_insert(documents)
table.query("ALTER TABLE documents COMPACT")

## Retrieve by Hybrid Search

This time, we not only get the relevant documents by Cosine Distance of Vectors, but also the BM25 scores. Then we using fusion method to merge the result set.

But you don't need to write it, `pytidb` already did it for you. You only need to search it by using `search_type="hybrid"` and point out the `text_column` like:



In [ ]:
question = "What is TiKV?"

results = table.search(question, search_type="hybrid").text_column("text").limit(1)
results.to_pandas()

## Generate the Answer

In [ ]:
from litellm import completion

llm_model = "bedrock/us.amazon.nova-lite-v1:0"

messages = [
    {"role": "system", "content": f"Please carefully answer the question by {str(results)}"},
    {"role": "user", "content": question}
]

llm_response = completion(
    model=llm_model,
    messages=messages,
)

print(llm_response.choices[0].message.content)